<a href="https://colab.research.google.com/github/shaankhurshid/bmi707/blob/pre-processing/Pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
# Copying the data from dropbox: copy links using wget here.
! wget 

In [11]:
# Importing the data
colnames = pd.array(range(1,112321)) # The number of time intervals in the database
train_controls = pd.read_csv('train_controls_transformed.csv?dl=0', names = colnames, header = None)
train_cases = pd.read_csv('train_cases_transformed.csv?dl=0', names = colnames, header = None)
#holdout_controls = pd.read_csv('holdout_controls_transformed.csv?dl=0', header = None)
#holdout_cases = pd.read_csv('holdout_cases_transformed.csv?dl=0', header = None)

KeyboardInterrupt: ignored

In [0]:
# Creating labels
zeros = pd.Series(np.zeros(694))
ones = pd.Series(np.ones(694))
y_train = zeros.append(ones)

In [0]:
# Combine cases and controls
x_train = pd.concat([train_controls, train_cases])

In [0]:
# Downsampling
x_train = x_train.groupby(np.arange(len(x_train.columns))//32, axis=1).mean()

In [0]:
# Re-shaping data
x_train_reshaped = x_train.values.reshape(-1, 1, 3510)


In [104]:
# Check that sizes make sense
print(train_controls.shape)
print(train_cases.shape)
print(x_train.shape)
print(y_train.shape)
print(x_train_reshaped.shape)

(694, 112320)
(694, 112320)
(1388, 3510)
(1388,)
(1388, 1, 3510)


In [119]:
# Training the RNN with LSTM for binary classification
import tensorflow as tf
from tensorflow.keras import Sequential,Model
model = tf.keras.Sequential()
model.add(layers.LSTM(128, input_shape=(1,3510)))
model.add(layers.Dense(2))
model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_42 (LSTM)               (None, 128)               1863168   
_________________________________________________________________
dense_22 (Dense)             (None, 2)                 258       
Total params: 1,863,426
Trainable params: 1,863,426
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])
model.fit(x_train_reshaped, y_train, epochs = 10, verbose = 1)